In [ ]:
!pip install azure-ai-textanalytics==5.2.0b1
!pip install bs4
!pip install requests

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# authenticate azure text analytics client
key = "677e7a68d2f94932a97572bfd9505ea3"
endpoint = "https://covid-corruption-nlp2.cognitiveservices.azure.com/"

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [ ]:
# opinion mining on trustpilot reviews


grouped_details = pd.read_csv('./datasets/grouped_details.csv')
link_cols = [x for x in grouped_details.columns if 'company_link' in x]

def get_reviews(x):
    results = []
    for cell in x:
        if pd.isna(cell) or not cell:
            results.append('')
        else:
            page = requests.get("https://uk.trustpilot.com/review/"+cell)
            soup = BeautifulSoup(page.content, "html.parser")
            reviewcontent = soup.find_all('p', {'class', 'review-content__text'})
            
            results.append([review.get_text().rstrip() for review in reviewcontent if review])
    return results

link_details = pd.DataFrame(grouped_details[link_cols])
tr_reviews = link_details.apply(get_reviews)
tr_reviews['company_name'] = grouped_details['company_name']

tr_reviews.to_csv('./datasets/reviews.csv', index=False)

In [ ]:
# 
tr_reviews =  pd.read_csv('./datasets/reviews/reviews.csv')
review_cols = [x for x in tr_reviews.columns if 'company_link' in x]

def opinion_mining(client, documents):
    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    negative_keywords=[]
    for document in doc_result:
        for sentence in document.sentences:
            for mined_opinion in sentence.mined_opinions:
                if mined_opinion.target.sentiment == 'negative':
                    negative_keywords.append(mined_opinion.assessments[0].text)
                else:
                    negative_keywords.append('')
    return negative_keywords 

def review_keywords(x):
    results = []
    for cell in x:
        words = ''
        if type(cell) == str:
            review = eval(cell)
            num_docs = len(review)

            for i in range(0, num_docs, 10):
                words += ' '.join(opinion_mining(
                    client, 
                    documents=review[i: min(num_docs, i+10)])
                )

        results.append(words)
    return results


review_details = pd.DataFrame(tr_reviews[link_cols])
review_keywords = review_details.apply(review_keywords)
review_keywords['company_name'] = tr_reviews['company_name']

review_keywords.to_csv('./datasets/tp_keywords.csv', index=False)

In [ ]:
review_keywords = pd.read_csv('./datasets/tp_keywords.csv')
review_keywords = review_keywords.drop(review_keywords.columns[0], axis=1)

review_keywords['company_link_0'] + review_keywords['company_link_1']